In [1]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
#Cargar archivo csv desde seaborn
df= pd.read_csv('Base_Paris_LIMPIA.csv')
df.head(20)

,Unnamed: 0,id,scrape_id,host_id,latitude,longitude,listing_url,last_scraped,source,name,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,reviews_per_month
0,0,3109,20240906025355,3631,48.831910,2.318700,https://www.airbnb.com/rooms/3109,2024-09-11,city scrape,zen and calm,...,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.0,1.0,0.05
1,1,5396,20240906025355,7903,48.852470,2.358350,https://www.airbnb.com/rooms/5396,2024-09-13,city scrape,Your perfect Paris studio on Île Saint-Louis,...,4.61,4.64,4.59,4.81,4.84,4.96,4.59,1.0,1.0,2.23
2,2,7397,20240906025355,2626,48.859090,2.353150,https://www.airbnb.com/rooms/7397,2024-09-06,city scrape,MARAIS - 2ROOMS APT - 2/4 PEOPLE,...,4.73,4.81,4.45,4.92,4.89,4.93,4.74,1.0,1.0,2.20
3,3,7964,20240906025355,22155,48.874170,2.342450,https://www.airbnb.com/rooms/7964,2024-09-10,previous scrape,Sunny apartment with balcony,...,4.80,5.00,5.00,5.00,5.00,5.00,5.00,1.0,1.0,0.03
4,4,241715,20240906025355,3342097,48.893464,2.378341,https://www.airbnb.com/rooms/241715,2024-09-11,city scrape,Big Cosy Appartement with 100 m2 Terrace in Paris,...,4.80,4.80,4.70,4.90,4.90,4.90,4.70,1.0,1.0,0.80
5,5,243040,20240906025355,1275649,48.871140,2.326060,https://www.airbnb.com/rooms/243040,2024-09-10,previous scrape,Apart Hotel / Paris Opera Madeleine,...,4.45,4.53,4.41,4.51,4.51,4.90,4.59,2.0,2.0,0.33
6,6,243245,20240906025355,1276616,48.880540,2.331350,https://www.airbnb.com/rooms/243245,2024-09-10,previous scrape,Charming studio in Paris-Montmartre,...,4.02,4.80,4.70,4.53,4.90,4.90,4.22,1.0,1.0,1.37
7,7,243961,20240906025355,1280100,48.873074,2.350599,https://www.airbnb.com/rooms/243961,2024-09-10,previous scrape,Beautifull loft - Grands Boulevards,...,4.59,4.77,4.74,4.41,4.90,4.70,4.65,1.0,1.0,1.02
8,8,9359,20240906025355,28422,48.860060,2.348630,https://www.airbnb.com/rooms/9359,2024-09-13,city scrape,"Cozy, Central Paris: WALK or VELIB EVERYWHERE !",...,4.80,4.80,4.70,4.90,4.90,4.90,4.70,1.0,1.0,0.80
9,9,9952,20240906025355,33534,48.863730,2.370930,https://www.airbnb.com/rooms/9952,2024-09-13,city scrape,Paris petit coin douillet,...,4.92,4.98,4.88,4.98,4.86,4.86,4.90,1.0,1.0,0.36


In [3]:
df.isnull().sum()

Unnamed: 0                                     0
id                                             0
scrape_id                                      0
host_id                                        0
latitude                                       0
                                              ..
review_scores_location                         0
review_scores_value                            0
calculated_host_listings_count                 0
calculated_host_listings_count_entire_homes    0
reviews_per_month                              0
Length: 72, dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95461 entries, 0 to 95460
Data columns (total 72 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Unnamed: 0                                   95461 non-null  int64  
 1   id                                           95461 non-null  int64  
 2   scrape_id                                    95461 non-null  int64  
 3   host_id                                      95461 non-null  int64  
 4   latitude                                     95461 non-null  float64
 5   longitude                                    95461 non-null  float64
 6   listing_url                                  95461 non-null  object 
 7   last_scraped                                 95461 non-null  object 
 8   source                                       95461 non-null  object 
 9   name                                         95461 non-null  object 
 10

In [5]:
#Sustituir valores nulos por un string en  concreto 
df["host_response_rate"] =df["host_response_rate"].replace('No captured',0) 
# Elimina cualquier símbolo no numérico (por ejemplo, '%') y convierte a tipo float
df['host_response_rate'] = df['host_response_rate'].str.replace('%', '').astype(float)

#Sustituir valores nulos por un string en  concreto 
df["host_acceptance_rate"] =df["host_acceptance_rate"].replace('No captured',0) 
# Elimina cualquier símbolo no numérico (por ejemplo, '%') y convierte a tipo float
df['host_acceptance_rate'] = df['host_acceptance_rate'].str.replace('%', '').astype(float)

#Realizamos una copia del dataframe
data=df.copy()

#Sustituir valores nulos con promedio o media
df["host_response_rate"]=df["host_response_rate"].fillna(round(data["host_response_rate"].mean(),1))
df["host_acceptance_rate"]=df["host_acceptance_rate"].fillna(round(data["host_acceptance_rate"].mean(),1))

In [6]:
#Rellenamos nulos
df =df.fillna(method="bfill")
df =df.fillna(method="ffill")

/var/folders/_8/d9swt24d6_5dqvpym3xln2ww0000gn/T/ipykernel_22142/1655763348.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df =df.fillna(method="bfill")
/var/folders/_8/d9swt24d6_5dqvpym3xln2ww0000gn/T/ipykernel_22142/1655763348.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df =df.fillna(method="ffill")


***"Variables categóricas"***

In [7]:
# Reemplazar los valores en la columna 'room_type'
df['room_type'] = df['room_type'].replace(["Entire home/apt"], "Home type")
df['room_type'] = df['room_type'].replace(["Private room", "Hotel room", "Shared room"], "Room type")

# Redefinimos las variables independientes (X) y dependiente (y)
Vars_Indep = df[['bathrooms', 'bedrooms', 'beds']]
Var_Dep = df['room_type']  # Asegúrate de usar 'room_type' con minúsculas

# Asignamos las variables
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

# Se escalan los datos
escalar = StandardScaler()

# Aplicamos el escalado a los datos de entrenamiento y de prueba
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar: Regresión Logística
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo con los datos de entrenamiento
algoritmo.fit(X_train, y_train)

# Realizamos la predicción sobre los datos de prueba
y_pred = algoritmo.predict(X_test)

# Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# Calculamos la precisión del modelo (ajusta pos_label para que coincida con tus clases)
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="Home type")  # o "Room type"
print('Precisión del modelo:')
print(precision)

# Calculamos la exactitud (accuracy) del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

# Calculamos la sensibilidad del modelo (ajusta pos_label también)
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Home type")  # o "Room type"
print('Sensibilidad del modelo:')
print(sensibilidad)

Matriz de Confusión:
[[17038     0]
 [ 2055     0]]
Precisión del modelo:
0.8923689310218405
Exactitud del modelo:
0.8923689310218405
Sensibilidad del modelo:
1.0


In [8]:
# Reemplazar los valores en la columna 'instant_bookable'

# Redefinimos las variables independientes (X) y dependiente (y)
Vars_Indep = df[['minimum_nights', 'maximum_nights','availability_365']]
Var_Dep = df['instant_bookable']  # Asegúrate de usar 'room_type' con minúsculas

# Asignamos las variables
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

# Se escalan los datos
escalar = StandardScaler()

# Aplicamos el escalado a los datos de entrenamiento y de prueba
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar: Regresión Logística
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo con los datos de entrenamiento
algoritmo.fit(X_train, y_train)

# Realizamos la predicción sobre los datos de prueba
y_pred = algoritmo.predict(X_test)

# Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# Calculamos la precisión del modelo (ajusta pos_label para que coincida con tus clases)
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="t")  # o "Room type"
print('Precisión del modelo:')
print(precision)

# Calculamos la exactitud (accuracy) del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

# Calculamos la sensibilidad del modelo (ajusta pos_label también)
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="t")  # o "Room type"
print('Sensibilidad del modelo:')
print(sensibilidad)

Matriz de Confusión:
[[14337    93]
 [ 4370   293]]
Precisión del modelo:
0.7590673575129534
Exactitud del modelo:
0.7662494107788195
Sensibilidad del modelo:
0.06283508470941454


In [9]:
#Variable dicotómica (host_response_time)

df['host_response_time']= df['host_response_time'].replace(["within an hour","within a few hours","within a day","a few days or more"], "Fast response")

# Redefinimos las variables independientes (X) y dependiente (y)
Vars_Indep = df[['review_scores_communication', 'reviews_per_month']]
Var_Dep = df['host_response_time']  
# Asignamos las variables
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

# Se escalan los datos
escalar = StandardScaler()

# Aplicamos el escalado a los datos de entrenamiento y de prueba
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar: Regresión Logística
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo con los datos de entrenamiento
algoritmo.fit(X_train, y_train)

# Realizamos la predicción sobre los datos de prueba
y_pred = algoritmo.predict(X_test)

# Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# Calculamos la precisión del modelo (ajusta pos_label para que coincida con tus clases)
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="Fast response")  # o "Room type"
print('Precisión del modelo:')
print(precision)

# Calculamos la exactitud (accuracy) del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

# Calculamos la sensibilidad del modelo (ajusta pos_label también)
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Fast response")  # o "Room type"
print('Sensibilidad del modelo:')
print(sensibilidad)

Matriz de Confusión:
[[8260 2664]
 [4589 3580]]
Precisión del modelo:
0.642851583780839
Exactitud del modelo:
0.6201225580055518
Sensibilidad del modelo:
0.7561332845111681


In [10]:
#Variable dicotómica (host_response_time)


df = df[df['host_is_superhost'] != 'No captured']
df['host_is_superhost']=df['host_is_superhost'].replace(['t'],'True')
df['host_is_superhost']=df['host_is_superhost'].replace(['f'],'False')
# Redefinimos las variables independientes (X) y dependiente (y)
Vars_Indep = df[['availability_30', 'host_total_listings_count','review_scores_rating']]
Var_Dep = df['host_is_superhost']  
# Asignamos las variables
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

# Se escalan los datos
escalar = StandardScaler()

# Aplicamos el escalado a los datos de entrenamiento y de prueba
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar: Regresión Logística
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo con los datos de entrenamiento
algoritmo.fit(X_train, y_train)

# Realizamos la predicción sobre los datos de prueba
y_pred = algoritmo.predict(X_test)

# Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# Calculamos la precisión del modelo (ajusta pos_label para que coincida con tus clases)
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="False")  # o "Room type"
print('Precisión del modelo:')
print(precision)

# Calculamos la exactitud (accuracy) del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

# Calculamos la sensibilidad del modelo (ajusta pos_label también)
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="False")  # o "Room type"
print('Sensibilidad del modelo:')
print(sensibilidad)


Matriz de Confusión:
[[15469    91]
 [ 2983    89]]
Precisión del modelo:
0.8383373076089313
Exactitud del modelo:
0.8350150279089739
Sensibilidad del modelo:
0.9941516709511569


In [11]:
#Variable dicotómica (host_has_profile_pic)

df = df[df['host_has_profile_pic'] != 'No captured']
df['host_has_profile_pic']=df['host_has_profile_pic'].replace(['t'],'True')
df['host_has_profile_pic']=df['host_has_profile_pic'].replace(['f'],'False')

# Redefinimos las variables independientes (X) y dependiente (y)
Vars_Indep = df[['number_of_reviews','minimum_nights','maximum_nights']]
Var_Dep = df['host_has_profile_pic']  
# Asignamos las variables
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

# Se escalan los datos
escalar = StandardScaler()

# Aplicamos el escalado a los datos de entrenamiento y de prueba
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar: Regresión Logística
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

# Entrenamos el modelo con los datos de entrenamiento
algoritmo.fit(X_train, y_train)

# Realizamos la predicción sobre los datos de prueba
y_pred = algoritmo.predict(X_test)

# Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

# Calculamos la precisión del modelo (ajusta pos_label para que coincida con tus clases)
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="True")  # o "Room type"
print('Precisión del modelo:')
print(precision)

# Calculamos la exactitud (accuracy) del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

# Calculamos la sensibilidad del modelo (ajusta pos_label también)
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="True")  # o "Room type"
print('Sensibilidad del modelo:')
print(sensibilidad)

Matriz de Confusión:
[[    0   746]
 [    0 17885]]
Precisión del modelo:
0.959959207771993
Exactitud del modelo:
0.959959207771993
Sensibilidad del modelo:
1.0


***"Variables numéricas"***

In [12]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=df['price'].max()
Min=df['price'].min()
Limites= [Min, Max]
#Categorización de variables
#Declaramos 2 intervalos 
intervalos=np.linspace(9, 486, 3)
intervalos
#Creamos las categorías 
categorias= ["Min.Price", "Max.Price"]
#Finalmente creamos las categorías en la columna numérica
df['price']=pd.cut(x= df['price'], bins=intervalos, labels= categorias )
df['price']
df['price'].isnull().sum()
df=df.dropna(subset=['price'])
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['bathrooms', 'bedrooms', 'beds']]
Var_Dep= df['price']

#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep
#Dividimos el conjunto de datos en la parte de entretenimiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

#Se escalan todos los datos
escalar= StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_tranform
X_train= escalar.fit_transform(X_train)
X_test=escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Min.Price")
print('Precisión del modelo:')
print(precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Min.Price")
print('Sensibilidad del modelo:')
print(sensibilidad)


Matriz de Confusión
[[    0  2240]
 [    0 16391]]
Precisión del modelo:
0.879770275347539
Exactitud del modelo:
0.879770275347539
Sensibilidad del modelo:
1.0


In [13]:
df['reviews_per_month'].isnull().sum()
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=df['reviews_per_month'].max()
Min=df['reviews_per_month'].min()
Limites= [Min, Max]
Limites
#Categorización de variables
#Declaramos 2 intervalos 
intervalos=np.linspace(0.02, 3.36, 3)
intervalos
#Creamos las categorías 
categorias= ["Min.Rewiews", "Max.Rewiews"]

#Finalmente creamos las categorías en la columna numérica
df['reviews_per_month']=pd.cut(x= df['reviews_per_month'], bins=intervalos, labels= categorias )
df['reviews_per_month']

df['reviews_per_month'].isnull().sum()
df=df.dropna(subset=['reviews_per_month'])
df.info()
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['review_scores_cleanliness', 'review_scores_communication', 'review_scores_location', 'review_scores_value']]
Var_Dep= df['reviews_per_month']
#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entretenimiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

#Se escalan todos los datos
escalar= StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_tranform
X_train= escalar.fit_transform(X_train)
X_test=escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Min.Rewiews")
print('Precisión del modelo:')
print(precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Min.Rewiews")
print('Sensibilidad del modelo:')
print(sensibilidad)

<class 'pandas.core.frame.DataFrame'>
Index: 90803 entries, 0 to 95460
Data columns (total 72 columns):
 #   Column                                       Non-Null Count  Dtype   
---  ------                                       --------------  -----   
 0   Unnamed: 0                                   90803 non-null  int64   
 1   id                                           90803 non-null  int64   
 2   scrape_id                                    90803 non-null  int64   
 3   host_id                                      90803 non-null  int64   
 4   latitude                                     90803 non-null  float64 
 5   longitude                                    90803 non-null  float64 
 6   listing_url                                  90803 non-null  object  
 7   last_scraped                                 90803 non-null  object  
 8   source                                       90803 non-null  object  
 9   name                                         90803 non-null  objec

In [14]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=df['price'].max()
Min=df['price'].min()
Limites= [Min, Max]
Limites

#Categorización de variables
#Declaramos 2 intervalos 
intervalos=np.linspace(9, 486, 3)
intervalos
categorias= ["Min.Price", "Max.Price"]
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['minimum_nights', 'maximum_nights']]
Var_Dep= df['price']
#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entretenimiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

#Se escalan todos los datos
escalar= StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_tranform
X_train= escalar.fit_transform(X_train)
X_test=escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Min.Price")
print('Precisión del modelo:')
print(precision)

#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Min.Price")
print('Sensibilidad del modelo:')
print(sensibilidad)

Matriz de Confusión
[[    0  2246]
 [    0 15915]]
Precisión del modelo:
0.8763283960134354
Exactitud del modelo:
0.8763283960134354
Sensibilidad del modelo:
1.0


In [15]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=df['availability_365'].max()
Min=df['availability_365'].min()
Limites= [Min, Max]
Limites
#Categorización de variables
#Declaramos 2 intervalos 
intervalos=np.linspace(0, 365, 3)
intervalos
#Creamos las categorías 
categorias= ["Inicio_Año", "Fin_Año"]

#Finalmente creamos las categorías en la columna numérica
df['availability_365']=pd.cut(x= df['availability_365'], bins=intervalos, labels= categorias )
df['availability_365']
df=df.dropna(subset=['availability_365'])
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['minimum_nights', 'maximum_nights']]
Var_Dep= df['availability_365']
#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entretenimiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

#Se escalan todos los datos
escalar= StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_tranform
X_train= escalar.fit_transform(X_train)
X_test=escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión')
print(matriz)
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Fin_Año")
print('Precisión del modelo:')
print(precision)
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Fin_Año")
print('Sensibilidad del modelo:')
print(sensibilidad)

Matriz de Confusión
[[2615 3685]
 [1987 4432]]
Precisión del modelo:
0.5682312038244242
Exactitud del modelo:
0.554052991587389
Sensibilidad del modelo:
0.4150793650793651


In [16]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=df['host_total_listings_count'].max()
Min=df['host_total_listings_count'].min()
Limites= [Min, Max]
Limites
#Categorización de variables
#Declaramos 2 intervalos 
intervalos=np.linspace(1, 11, 3)
intervalos
#Creamos las categorías 
categorias= ["Min.Count", "Max.Count"]

#Finalmente creamos las categorías en la columna numérica
df['host_total_listings_count']=pd.cut(x= df['host_total_listings_count'], bins=intervalos, labels= categorias )
df['host_total_listings_count']
df=df.dropna(subset=['host_total_listings_count'])
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['latitude', 'longitude', 'availability_30']]
Var_Dep= df['host_total_listings_count']
#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

#Dividimos el conjunto de datos en la parte de entretenimiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

#Se escalan todos los datos
escalar= StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_tranform
X_train= escalar.fit_transform(X_train)
X_test=escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión')
print(matriz)
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="Min.Count")
print('Precisión del modelo:')
print(precision)
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Min.Count")
print('Sensibilidad del modelo:')
print(sensibilidad)

Matriz de Confusión
[[   0  542]
 [   0 6615]]
Precisión del modelo:
0.9242699455078943
Exactitud del modelo:
0.9242699455078943
Sensibilidad del modelo:
1.0
